## Set up

In [ ]:
# # SETUP 
import os
import sys

# # Set Java (SỬA PATH NÀY!)
os.environ['JAVA_HOME'] = 'C:\\Java\\jdk-1.8'

# # QUAN TRỌNG: Bypass Hadoop requirement
os.environ['HADOOP_HOME'] = os.environ.get('JAVA_HOME')
os.environ['PATH'] = f"{os.environ['JAVA_HOME']}\\bin;{os.environ.get('PATH', '')}"

print(f"JAVA_HOME: {os.environ['JAVA_HOME']}")

In [ ]:
# INSTALL FINDSPARK
!pip install pyspark findspark -q

In [ ]:
# IMPORT LIBRARIES
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from functools import reduce
import warnings
warnings.filterwarnings('ignore')

print("Imports successful")

## Khởi tạo Spark Session

In [ ]:
# INITIALIZE SPARK SESSION
import tempfile

spark = SparkSession.builder \
    .appName("YouTubePreprocessing") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .config("spark.sql.warehouse.dir", tempfile.gettempdir()) \
    .config("spark.ui.enabled", "false") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
print(f"Spark {spark.version} started")

## Đọc dữ liệu

In [ ]:
raw_df = spark.read.csv("./data/raw_data.csv", header=True, inferSchema=True)

# Kiểm tra format trending_date để hiểu dữ liệu
print("=== SAMPLE TRENDING_DATE VALUES ===")
raw_df.select("trending_date").filter(col("trending_date").isNotNull()).distinct().show(10, False)

print("RAW DATA OVERVIEW")
raw_df.show(5)

print("VALID VIDEO ROWS (có video_id)")
valid_videos = raw_df.filter(col("video_id").isNotNull() & (col("video_id") != ""))
print(f"Valid videos: {valid_videos.count()} / {raw_df.count()}")
valid_videos.show(5)

## Kiểm tra các giá trị trending_date không hợp lệ

In [ ]:
print("INVALID TRENDING_DATE VALUES")
raw_df.select("trending_date").filter(
    col("trending_date").isNotNull() & 
    ~col("trending_date").rlike(r"^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z$")
).distinct().show(20, False)

print("COUNT COMPARISON")
valid_dates = raw_df.filter(col("trending_date").rlike(r"^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z$")).count()
total_with_dates = raw_df.filter(col("trending_date").isNotNull()).count()
print(f"Valid dates: {valid_dates} / {total_with_dates}")

## Khai báo hàm dataframe_info

In [ ]:
# Helper function
def dataframe_info(df):
    print(f"{'-'*40}")
    print(f"Số dòng: {df.count()}, Số cột: {len(df.columns)}")
    print(f"{'-'*40}")
    df.printSchema()
    print(f"{'-'*40}")
    df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

dataframe_info(raw_df)

## Tiền xử lý dữ liệu

### 1, Xóa các cột không cần thiết

In [ ]:
preprocessed_data = raw_df.drop('thumbnail_link', 'comments_disabled', 'video_error_or_removed', 'ratings_disabled')
dataframe_info(preprocessed_data)

### 2, Xóa các hàng có tất cả giá trị là Null

In [ ]:
preprocessed_data = preprocessed_data.filter(
    reduce(lambda a, b: a | b, (col(c).isNotNull() for c in preprocessed_data.columns))
)
dataframe_info(preprocessed_data)

In [ ]:
dataframe_info(preprocessed_data)

### 3, Xóa các hàng có trending_date sai định dạng

In [ ]:
# Lọc dữ liệu video hợp lệ - chỉ giữ những dòng có trending_date đúng format
print("Before filtering:")
print(f"Total rows: {preprocessed_data.count()}")

# Lọc chỉ những dòng có trending_date đúng format ISO timestamp
preprocessed_data = preprocessed_data.filter(
    col("trending_date").rlike(r"^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z$")
)

print("After filtering:")
print(f"Valid rows: {preprocessed_data.count()}")
dataframe_info(preprocessed_data)

### 4, Điền giá trị Null cho description

In [ ]:
preprocessed_data = preprocessed_data.fillna({"description": "No description"})
dataframe_info(preprocessed_data)

### 5, Chuẩn hóa dữ liệu

In [ ]:
# Chuyển đổi timestamp - SỬA FORMAT CHO ĐÚNG
preprocessed_data = preprocessed_data.withColumn('trending_date', to_timestamp('trending_date', "yyyy-MM-dd'T'HH:mm:ss'Z'"))
preprocessed_data = preprocessed_data.withColumn('publishedAt', to_timestamp('publishedAt', "yyyy-MM-dd'T'HH:mm:ss'Z'"))

# Dataset riêng cho machine learning
ML_data = preprocessed_data
ML_data = ML_data.withColumn('tags', when(col('tags') == '[none]', '').otherwise(col('tags')))
ML_data = ML_data.withColumn('tags', split(regexp_replace('tags', '"', ''), '\\|'))

preprocessed_data.show(10)

In [ ]:
dataframe_info(preprocessed_data)

## Lưu dữ liệu đã xử lý

In [ ]:
# Convert timestamps to string để tránh lỗi khi save
preprocessed_save = preprocessed_data.withColumn('trending_date', 
    date_format('trending_date', 'yyyy-MM-dd HH:mm:ss')) \
    .withColumn('publishedAt', 
    date_format('publishedAt', 'yyyy-MM-dd HH:mm:ss'))

ML_save = ML_data.withColumn('trending_date', 
    date_format('trending_date', 'yyyy-MM-dd HH:mm:ss')) \
    .withColumn('publishedAt', 
    date_format('publishedAt', 'yyyy-MM-dd HH:mm:ss'))

# Lưu files
preprocessed_save.toPandas().to_csv('./data/preprocessed_data.csv', index=False)
ML_save.toPandas().to_csv('./data/ml_data.csv', index=False)
print("Dữ liệu đã được lưu")

In [ ]:
# Stop Spark
spark.stop()